In [1]:
import pandas as pd
import xarray as xr
import numpy as np

import re
import datetime
import time

import os

import matplotlib.pyplot as plt
import matplotlib

from sys import getsizeof,path
path.append("../src")
from Utility import DateFrom8digitName,RKI_AG_to_int

In [36]:
def NRW_ageGroups(fn):
    df = pd.read_csv(fn,sep="\t",header=[1],thousands='.',decimal=",",na_values="-")
    
    df = df.filter(["Fälle","Fälle.1","Fälle.2","Todesfälle","Todesfälle.1","Todesfälle.2"])
    df = df.fillna(0).iloc[:-2]
    df = df.astype("int64")
    df.index.name="age"
    df.index = df.index.map(RKI_AG_to_int)
    
    f = {"Fälle.1":"male","Fälle.2":"female","Todesfälle.1":"male","Todesfälle.2":"female","Fälle":"total","Todefälle":"total"}
    df = df.rename(columns=f)
#    df["divers"] = df["total"]-(df["male"]+df["female"])
    
    return df.to_xarray().to_array()

def ImportNRW(d="../Data/Bundeslaender/NRW"):
    files = sorted(os.listdir(d))

    Altersgruppe = {}
    Woche = {}

    for fn in files[:]:
        dt = DateFrom8digitName(d+"/"+fn)
        dfn = d+"/"+fn
        if dt != None:
            fid = fn.split("_")[2]
            if fid == "Cases7dSum":
                pass
            elif fid == "CasesAGs":
                NRW_ageGroups(dfn)
            elif fid == "VerstorbeneAGs":
                NRW_ageGroups(dfn)
            elif fid == "Verstorbene":
                pass
            else:
                print(fn,dt)
            
        
t0 = time.time()

ImportNRW()
        
t1 = time.time()